# Atlassian Toolkit

This notebook demonstrates how to use the Atlassian Toolkit, including detailed examples for interacting with the Jira and Confluence APIs. This toolkit is designed to be extensible and will allow future integrations with other tools in the Atlassian suite, such as Crowd, BitBucket, Bamboo, Jira Service Desk, Xray, and Cloud Admin.

## Requirements

Before starting, make sure to have the following environment variables set:

- `ATLASSIAN_API_TOKEN`
- `ATLASSIAN_USERNAME`
- `ATLASSIAN_INSTANCE_URL`
- `ATLASSIAN_CLOUD`

### Installing Dependencies

First, let's install the necessary dependencies:

In [ ]:
%pip install atlassian-python-api
%pip install langchain-community
%pip install langchain-openai

### Environment Setup

Set the required environment variables for authentication:

In [ ]:
import os

os.environ["ATLASSIAN_API_TOKEN"] = "your_token_here"
os.environ["ATLASSIAN_USERNAME"] = "your_username_here"
os.environ["ATLASSIAN_INSTANCE_URL"] = "https://your_instance.atlassian.net"
os.environ["ATLASSIAN_CLOUD"] = "True"
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

### Initializing the Toolkit

In [ ]:
from langchain_community.utilities.atlassian import AtlassianAPIWrapper
from langchain_community.agent_toolkits.atlassian.toolkit import AtlassianToolkit
from langchain.agents import create_react_agent
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

# Initializing the LLM
llm = OpenAI(temperature=0.5)

# Initializing the Atlassian API Wrapper
atlassian = AtlassianAPIWrapper()

# Creating the Toolkit
toolkit = AtlassianToolkit.from_atlassian_api_wrapper(atlassian)

# Define the prompt for the agent
prompt_template = PromptTemplate(
    input_variables=["input", "tools", "tool_names", "agent_scratchpad"],
    template="""
You are an assistant helping to manage Atlassian services. You can interact with Jira and Confluence to perform various tasks like creating issues, running JQL/CQL queries, and retrieving available functions.

Given a task, determine the appropriate tool to use and execute the necessary actions.

{input}

Tools available:
{tools}

Tool names: {tool_names}

Agent scratchpad: {agent_scratchpad}

Your response should be in the following format:
Action: <tool_name>
Action Input: <input_for_tool>
"""
)

# Initializing the Agent
agent = create_react_agent(llm, tools=toolkit.get_tools(), prompt=prompt_template)


## Usage Examples

### Creating an Issue in Jira

Let's create a new issue in Jira:

In [ ]:
task_input = "Create a new issue in project CS with summary 'Test Summary' and description 'Test Description for creating an issue in Jira'"
task = {
    "input": task_input,
    "tools": toolkit.get_tools(),
    "tool_names": [tool.name for tool in toolkit.get_tools()],
    "agent_scratchpad": [],
    "intermediate_steps": []
}

response = agent.invoke(task)
print(response)

### Running a JQL Query in Jira

Let's run a JQL query in Jira to fetch issues in project CS:

In [ ]:
query = "project = CS AND status = Open"
response = atlassian.run("jira_jql", query)
print(response)

### Getting Jira Functions

We can list all available functions in Jira:

In [ ]:
response = atlassian.run("jira_get_functions", "")
print(response)

### Getting Parameters of a Jira Function

Get parameters of a specific Jira function, for example, `create_issue`:

In [ ]:
function_name = "create_issue"
response = atlassian.run("jira_get_functions", function_name)
print(response)

### Creating an Issue using Jira API Method

Create an issue using a specific Jira API method with detailed fields:

In [ ]:
import json

issue_create_dict = json.dumps(
    {
        "function": "create_issue",
        "args": [],
        "kwargs": {
            "fields": {
                "summary": "Automated Issue",
                "description": "Issue created via API call",
                "issuetype": {"name": "Bug"},
                "project": {"key": "CS"},
            }
        },
    }
)
response = atlassian.run("jira_other", issue_create_dict)
print(response)

### Getting Confluence Functions

We can list all available functions in Confluence:

In [ ]:
response = atlassian.run("confluence_get_functions", "")
print(response)

### Getting Parameters of a Confluence Function

Get parameters of a specific Confluence function, for example, `get_page_id`:

In [ ]:
function_name = "get_page_id"
response = atlassian.run("confluence_get_functions", function_name)
print(response)

### Getting a Page ID using Confluence API Method

Get a page ID using a specific Confluence API method:

In [ ]:
get_page_dict = json.dumps(
    {
        "function": "get_page_id",
        "args": [],
        "kwargs": {"space": "MPE", "title": "Test Page", "type": "page"},
    }
)
response = atlassian.run("confluence_other", get_page_dict)
print(response)

### Running a CQL Query in Confluence

Run a CQL query to fetch pages in Confluence:

In [ ]:
query = "type=page"
response = atlassian.run("confluence_cql", query)
print(response)

### Applying Filters to Response

Apply filters to the response data to remove sensitive information:

In [ ]:
atlassian.filter_keys = ["*password*"]
response = {"username": "user1", "password": "secret"}
filtered_response = atlassian.apply_filter(response)
print(filtered_response)

## Future Integrations
This module is extensible and will soon support new modules from the Atlassian tool suite, including:

- Crowd module
- BitBucket module
- Bamboo module
- Jira Service Desk module
- Xray module
- Cloud Admin module

The Atlassian toolkit supports 100% of all functions currently available in the `atlassian-python-api` library, allowing you to extend it to all products in the Atlassian suite. For more information, see the [Atlassian Python API documentation](https://atlassian-python-api.readthedocs.io/).
